# Support vector machines solvers

The objective is to solve the **primal** problem:

$$
\begin{aligned}
\min_{w \in \mathbb{R}^n,\, z \in \mathbb{R}^m}  & \quad \frac{1}{2} ||w||_2^2 + C \mathbb{1}^T z \\
\text{subject to } & \quad y_i(w^T x_i) \geq 1-z_i, \quad i = 1,...,m \\
& \quad z \geq 0
\end{aligned}
$$

We define $f(w, z) = \frac{1}{2} ||w||_2^2 + C \mathbb{1}^T z$ the objective function and, for $i = 1,...,m$, $g_i(w, z) = 1-z_i-y_i(w^T x_i)$ and $h_i(w, z) = -z_i$ the inequality constraint functions.

We can rewrite the optimization problem using these functions:

$$
\begin{aligned}
\min_{w,\, z} & \quad f(w, z) \\
\text{subject to } & \quad g_i(w, z) \leq 0, \quad i = 1,...,m \\
& \quad h_i(w, z) \leq 0, \quad i = 1,...,m
\end{aligned}
$$

**Note :** A vector $p \in \mathbb{R}^k$ is represented by a one-column matrix with $k$ lines. For making notations easy to write, we represent the vertical concatenation of $p \in \mathbb{R}^k$ and $q \in \mathbb{R}^l$ by $[p \, |\, q]$.

## Primal and dual approximations via logarithmic barrier

### Primal approximation

We approximate the primal problem by this problem:

$$
\begin{aligned}
\min_{w,\, z} & \quad f(w, z) - \frac{1}{t}\left(\sum_{i = 1}^m \log(-g_i(w, z)) +  \sum_{i = 1}^m \log(-h_i(w, z))\right)
\end{aligned}
$$

To optimize it, we will need the following gradients:

- $\nabla f(w, z) = [w\,|\,C\mathbb{1}]$
- $\nabla g_i(w, z) = -[y_i x_i\,|\,\mathbb{1}_i]$ where $\mathbb{1}_i$ is the vector with a $1$ at position $i$ and zeros otherwise
- $\nabla h_i(w, z) = -[\mathbb{0}\,|\,\mathbb{1}_i]$

and the following hessians:

- $\nabla^2 f(w, z) = \left( \begin{array}{cc}
I_n & O \\
O & O \end{array} \right)$
- $\nabla^2 g_i(w, z) = O$
- $\nabla^2 h_i(w, z) = O$

### Dual approximation

Before approximating the dual, we need to find the **dual** problem. We define the laplacian:
$$L(w, z, u, v) = f(w, z) + \sum_{i = 1}^m u_i g_i(w, z) + \sum_{i = 1}^m v_i h_i(w, z).$$
As $\nabla^2_{w, z} L(w, z, u, v) = \left( \begin{array}{cc}
I_n & O \\
O & O \end{array} \right) \succcurlyeq 0$, $(w, z) \mapsto L(w, z, u, v)$ is convex. In order to minimize $L(w, z, u, v)$ in $w$ and $z$, we just need to find $w$ and $z$ such that:

- $\nabla_w L(w, z, u, v) = w - \sum_{i = 1}^{m} u_i y_i x_i = \mathbb{0}$
- $\nabla_z L(w, z, u, v) = C \mathbb{1} - u - v = \mathbb{0}$.

Then, we get the dual problem:

$$
\begin{aligned}
\max_{u \in \mathbb{R}^m}  & \quad - \frac{1}{2} ||Ku||_2^2 + \mathbb{1}^T u \\
\text{subject to } & \quad -u \leq 0 \\
& \quad u - C\mathbb{1}\leq 0
\end{aligned}
$$

where $K = \left( y_1 x_1 \,|\, ... \,|\, y_m x_m \right)$.

We define $r(u) = - \frac{1}{2} ||Ku||_2^2 + \mathbb{1}^T u$ the objective function and, for $i = 1,...,m$, $s_i(u) = -u_i$ and $t_i(u) = u_i-C$ the inequality constraint functions.

We approximate the dual problem by this problem:

$$
\begin{aligned}
\min_{u} & \quad r(u) - \frac{1}{t}\left(\sum_{i = 1}^m \log(-s_i(w, z)) +  \sum_{i = 1}^m \log(-t_i(w, z))\right)
\end{aligned}
$$

To optimize it, we will need the following gradients:

- $\nabla r(u) = - K^T K u + \mathbb{1}$
- $\nabla s_i(u) = -\mathbb{1}_i$
- $\nabla t_i(u) = \mathbb{1}_i$

and the following hessians:

- $\nabla^2 r(u) = - K^T K$
- $\nabla^2 s_i(u) = O$
- $\nabla^2 t_i(u) = O$

## Primal and dual optimizations via logarithmic barrier

In [ ]:
import numpy

In [ ]:
def logarithmic_barrier_approx(t, Fs, Fps, Fpps):
    """
    Computes the logarithmic barrier approximation function
    of a convex optimization problem, its gradient and its hessian
    (formulas in slide 74, course 2).
    
    Args:
        t: the approximation quality
        Fs: the list with the objective function first and the
            inequality constraint functions next
        Fps: the list with the gradients of the objective function first
             and the inequality constraint functions next
        Fpps: the list with the hessians of the objective function first
              and the inequality constraint functions next
    
    Returns:
        The logarithmic barrier approximation function, its gradient
        and its hessian.
    """
    def LF(x):
        r = Fs[0](x)
        for i in range(1, len(Fs)):
            r += 1/t*(-np.log(-Fs[i](x)))
        return r
    
    def LFp(x):
        r = Fps[0](x)
        for i in range(1, len(Fs)):
            r += 1/t*(-1/Fs[i](x)*Fps[i](x))
        return r
    
    def LFpp(x):
        r = Fpps[0](x)
        for i in range(1, len(Fs)):
            r += 1/t*(1/Fs[i](x)**2*Fps[i](x).dot(Fps[i](x).T)-1/Fs[i](x)*Fpps[i](x)) 
        return r
    
    return LF, LFp, LFpp

In [ ]:
# Backtracking line search parameters
alpha = 1/2
beta = 3/4

def newton(F, Fp, Fpp, x0, eps):
    """
    Minimizes F using Newton's method (algorithm in slide 20, course 2).
    
    Args:
        F: a function from R^n to R
        Fp: the gradient of F
        Fpp: the hessian of F
        x0: the initial point
        eps: the precision of the approximation
    
    Returns:
        The sequence of approximations of the extremum of F up to a
        precision eps. Useful for debugging.
    """
    def get_next_x(x):
        Fpx = Fp(x)
        delta_x = -np.linalg.inv(Fpp(x)).dot(Fpx)

        # Backtracking line search
        t = 1
        while F(x + t*delta_x) >= F(x)+alpha*t*Fpx.T.dot(delta_x):
            t = beta*t

        return x + t*delta_x
    
    xn = [np.inf, x0]

    while np.linalg.norm(xn[-1] - xn[-2]) >= eps:
        xn.append(get_next_x(xn[-1]))
    
    return xn[1:]

In [ ]:
# Barrier method parameter
mu = 3

def barrier(Fs, Fps, Fpps, x0, eps):
    """
    Approximate the extremum of F using barrier method
    (algorithm in slide 80, course 2).
    
    Args:
        Fs: the list with the objective function first and the
            inequality constraint functions next
        Fps: the list with the gradients of the objective function first
             and the inequality constraint functions next
        Fpps: the list with the hessians of the objective function first
              and the inequality constraint functions next
        x0: the initial point
        eps: the precision of the approximation
    
    Returns:
        An approximation of the extremum of F.
    """
    x = x0
    t = 1
    m = len(Fs) - 1
    
    while m/t >= eps:
        LF, LFp, LFpp = logarithmic_barrier_approx(t, Fs, Fps, Fpps)
        x = newton(LF, LFp, LFpp, x, eps)[-1]
        t = mu*t
    
    return x